# Efficiency vs quality tradeoffs

Author: Monique Monteiro - moniquelouise@gmail.com



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [1]:
main_dir = "/content/gdrive/MyDrive/Unicamp-aula-10"

## Libraries installation

In [ ]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 101.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=e395c7562144b17daeb1f64c54ee50a00f8261d9c96eaa0a6c9a3120a65d1222
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

!pip install pyserini -q
!pip install faiss-cpu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 110.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 28.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 88.1 MB/s eta 0:00:00


In [ ]:
!pip install evaluate -q
!pip install trectools -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 36.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


## Passages download

In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz

--2023-05-06 21:33:47--  https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz
Resolving huggingface.co (huggingface.co)... 18.155.68.44, 18.155.68.121, 18.155.68.38, ...
Connecting to huggingface.co (huggingface.co)|18.155.68.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/a8/10/a810e88b0e7b233be82b89c1fa6ec2d75efc6d55784c2ada9dcac8434a634f3a/e9e97686e3138eaff989f67c04cd32e8f8f4c0d4857187e3f180275b23e24e85?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27corpus.jsonl.gz%3B+filename%3D%22corpus.jsonl.gz%22%3B&response-content-type=application%2Fgzip&Expires=1683668029&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2E4LzEwL2E4MTBlODhiMGU3YjIzM2JlODJiODljMWZhNmVjMmQ3NWVmYzZkNTU3ODRjMmFkYTlkY2FjODQzNGE2MzRmM2EvZTllOTc2ODZlMzEzOGVhZmY5ODlmNjdjMDRjZDMyZThmOGY0YzBkNDg1NzE4N2UzZjE4MDI3NWIyM2UyNGU4NT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSo

In [ ]:
!mv corpus.jsonl.gz {main_dir}/trec-covid

In [ ]:
!gunzip {main_dir}/trec-covid/corpus.jsonl.gz

gzip: /content/gdrive/MyDrive/Unicamp-aula-10/trec-covid/corpus.jsonl already exists; do you wish to overwrite (y or n)? ^C


## Queries vs passages download

In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv

--2023-05-06 21:34:54--  https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv
Resolving huggingface.co (huggingface.co)... 18.155.68.44, 18.155.68.116, 18.155.68.121, ...
Connecting to huggingface.co (huggingface.co)|18.155.68.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 980831 (958K) [text/plain]
Saving to: ‘test.tsv’

test.tsv            100%[===================>] 957.84K  1.09MB/s    in 0.9s    

2023-05-06 21:34:55 (1.09 MB/s) - ‘test.tsv’ saved [980831/980831]



In [ ]:
!mv test.tsv {main_dir}/trec-covid

## Queries download

In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/queries.jsonl.gz

--2023-05-06 21:34:56--  https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/queries.jsonl.gz
Resolving huggingface.co (huggingface.co)... 18.155.68.44, 18.155.68.116, 18.155.68.121, ...
Connecting to huggingface.co (huggingface.co)|18.155.68.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/a8/10/a810e88b0e7b233be82b89c1fa6ec2d75efc6d55784c2ada9dcac8434a634f3a/9eadcc2cdf140addc9dae83648bb2c6611f5e4b66eaed7475fa5a0ca48eda371?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27queries.jsonl.gz%3B+filename%3D%22queries.jsonl.gz%22%3B&response-content-type=application%2Fgzip&Expires=1683668096&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2E4LzEwL2E4MTBlODhiMGU3YjIzM2JlODJiODljMWZhNmVjMmQ3NWVmYzZkNTU3ODRjMmFkYTlkY2FjODQzNGE2MzRmM2EvOWVhZGNjMmNkZjE0MGFkZGM5ZGFlODM2NDhiYjJjNjYxMWY1ZTRiNjZlYWVkNzQ3NWZhNWEwY2E0OGVkYTM3MT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9

In [ ]:
!mv queries.jsonl.gz {main_dir}/trec-covid

In [ ]:
!gunzip {main_dir}/trec-covid/queries.jsonl.gz

gzip: /content/gdrive/MyDrive/Unicamp-aula-10/trec-covid/queries.jsonl already exists; do you wish to overwrite (y or n)? ^C


## Pipeline 1: Pyserini BM25 + Reranking with cross-encoder/ms-marco-MiniLM-L-6-v2

### 1st step: Pre-indexing TREC-COVID corpus

Here, we need to measure indexing cost in dollars per hour.

In [2]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda')

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

model_name = 'cross-encoder/ms-marco-MiniLM-L-6-v2'

embedder = SentenceTransformer(model_name, device=device)



Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/cross-encoder_ms-marco-MiniLM-L-6-v2 were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
import jsonlines

passage_ids = []
passage_texts = []
id_to_passage = dict()

#Maps each id to its position in the corpus
id_to_index = dict()

with jsonlines.open(f"{main_dir}/trec-covid/corpus.jsonl") as reader:
  for item in reader:
    id = item["_id"]
    id_to_index[id] = len(passage_ids)
    passage_ids.append(id)
    text = item["title"] + ' ' + item["text"]
    passage_texts.append(text)
    id_to_passage[id] = text

In [ ]:
%time corpus_embeddings = embedder.encode(passage_texts, convert_to_tensor=True)

CPU times: user 6min 29s, sys: 10.1 s, total: 6min 39s
Wall time: 3min


In [ ]:
corpus_embeddings.shape

torch.Size([171332, 384])

In [ ]:
%%writefile gpu_usage.sh
#! /bin/bash
#comment: run for 10 seconds, change it as per your use
end=$((SECONDS+3600))

while [ $SECONDS -lt $end ]; do
    nvidia-smi --format=csv --query-gpu=power.draw,utilization.gpu,memory.used,memory.free,fan.speed,temperature.gpu >> gpu.log
    #comment: or use below command and comment above using #
    #nvidia-smi dmon -i 0 -s mu -d 1 -o TD >> gpu.log
done

Overwriting gpu_usage.sh


In [ ]:
%%bash --bg

bash gpu_usage.sh

In [ ]:
corpus_embeddings = embedder.encode(passage_texts, convert_to_tensor=True) #2m41s

In [ ]:
corpus_embeddings.shape

torch.Size([171332, 384])

### Pipeline - Searching with Pyserini BM25 + Reranking

In [5]:
from pyserini.search.lucene import LuceneSearcher
import numpy as np
import json

In [6]:
def search_with_bm25(query,k = 1000):
  searcher = LuceneSearcher.from_prebuilt_index('beir-v1.0.0-trec-covid.flat')
  hits = searcher.search(query, k)
  return hits

In [7]:
query_ids = []
query_texts = []

with jsonlines.open(f"{main_dir}/trec-covid/queries.jsonl") as reader:
  for item in reader:
    id = item["_id"]
    query_ids.append(id)
    text = item["text"]
    query_texts.append(text)
     

In [ ]:
query_texts[0]

'what is the origin of COVID-19'

Generates the first request to download pre-indexed corpus.

In [ ]:
hits = search_with_bm25(query_texts[0])

lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.tar.gz: 216MB [00:20, 11.1MB/s]                           


In [ ]:
len(hits)

1000

In [ ]:
jsondoc = json.loads(hits[0].raw)
jsondoc

{'_id': 'dv9m19yk',
 'title': '[What is the origin of SARS-CoV-2?]',
 'text': 'Every time a pandemic occurs, dozens of theories emerge to attribute the origin of the event to different facts. The COVID-19 pandemic that has hit virtually all the globe has been no exception. What is known so far about the origin of the virus that causes COVID 19? The first investigations on the origin of this disease have determined that it is a new type of virus, the origin of which is most likely zoonotic.',
 'metadata': {'url': 'https://www.ncbi.nlm.nih.gov/pubmed/32412715/',
  'pubmed_id': '32412715'}}

In [ ]:
id_to_passage['dv9m19yk']

'[What is the origin of SARS-CoV-2?] Every time a pandemic occurs, dozens of theories emerge to attribute the origin of the event to different facts. The COVID-19 pandemic that has hit virtually all the globe has been no exception. What is known so far about the origin of the virus that causes COVID 19? The first investigations on the origin of this disease have determined that it is a new type of virus, the origin of which is most likely zoonotic.'

In [ ]:
id_to_index['dv9m19yk']

81848

In [ ]:
passage_ids[81848]

'dv9m19yk'

In [ ]:
passage_texts[81848]

'[What is the origin of SARS-CoV-2?] Every time a pandemic occurs, dozens of theories emerge to attribute the origin of the event to different facts. The COVID-19 pandemic that has hit virtually all the globe has been no exception. What is known so far about the origin of the virus that causes COVID 19? The first investigations on the origin of this disease have determined that it is a new type of virus, the origin of which is most likely zoonotic.'

In [ ]:
corpus_embeddings[81848].shape

torch.Size([384])

In [ ]:
def search(query, top_k=1000):
  #1st step: searches with BM25
  bm25_hits = search_with_bm25(query, k=top_k)

  ids = [json.loads(bm25_hits[i].raw)['_id'] for i in range(len(bm25_hits))]
  indices = [id_to_index[id] for id in ids]

  #Finds corpus embeddings
  passages_embeddings = [corpus_embeddings[i] for i in indices]

  query_embedding = embedder.encode(query, convert_to_tensor=True)

  hits = util.semantic_search(query_embedding, passages_embeddings, top_k=top_k)
  hits = hits[0]      #Get the hits for the first query
  results = dict()
  for hit in hits:
    assert hit['corpus_id'] < 1000
    results[ids[hit['corpus_id']]] = hit['score']

  return results

In [ ]:
%time search(query_texts[0])

CPU times: user 106 ms, sys: 4.99 ms, total: 111 ms
Wall time: 91.5 ms


{'gh07kf93': 0.9902833700180054,
 'ghuwz4w7': 0.9848959445953369,
 'x33dcjov': 0.9848238825798035,
 'rzpbpxw2': 0.9838396906852722,
 'w1jf3cdm': 0.9807307720184326,
 'gpy92gm5': 0.9806936383247375,
 'zumw6efw': 0.9798028469085693,
 '3l7du0jt': 0.9795253276824951,
 'wzi78q06': 0.979301929473877,
 'gdfxiosc': 0.9790084362030029,
 '6m70ltbw': 0.9777147769927979,
 'zb6h3q4k': 0.9776499271392822,
 'pt98rqyr': 0.9773953557014465,
 'cy3inhrd': 0.975878119468689,
 'nc38gr2f': 0.9755426645278931,
 'xwhxyy3d': 0.9755305051803589,
 'j23gik22': 0.9730088710784912,
 'b25m9pub': 0.9728070497512817,
 'ck8l0c6k': 0.9728070497512817,
 '57q0buzy': 0.969458818435669,
 '2fzd4dn4': 0.9687535762786865,
 '9puijlym': 0.9685115814208984,
 'm86s9az3': 0.967975914478302,
 'ewdow4b5': 0.9656345844268799,
 '8ymx739h': 0.965180516242981,
 'kybnap5u': 0.9646924138069153,
 'a93k6o3b': 0.9640289545059204,
 'lyxul945': 0.9638450741767883,
 '3h4z9970': 0.9630392789840698,
 '9duc0kjf': 0.9628820419311523,
 'iyb3nt4q': 0.

In [ ]:
id_to_passage['gh07kf93']

'What are the economic implications of COVID-19 '

In [ ]:
id_to_passage['ghuwz4w7']

'What the Diamond Princess taught the world about covid-19 '

In [ ]:
id_to_passage['x33dcjov']

'What the COVID-19 Crisis Is Telling Humanity '

In [ ]:
id_to_passage['xefpg0zo']

'What’s Important: Facing Fear in the Time of COVID-19 '

### Measuring NDCG@10

In [8]:

import pandas as pd

qrel = pd.read_csv(f"{main_dir}/trec-covid/test.tsv", sep="\t", header=None, 
                   skiprows=1, names=["query", "docid", "rel"])
qrel["q0"] = "q0"
qrel = qrel.to_dict(orient="list")

In [ ]:
from collections import defaultdict
from tqdm import tqdm

def search_all(top_k=1000):
  run = defaultdict(list)

  for id, query in tqdm(zip(query_ids, query_texts)):
    results = search(query)
    run["query"] += [id] * top_k
    run["docid"] += results.keys()
    run["score"] += results.values()
    run["q0"] += ["q0"] * top_k
    run["rank"] += list(range(1,top_k+1))
    run["system"] += [model_name] * top_k

  return run

In [ ]:
run = search_all()

50it [00:07,  6.52it/s]


In [ ]:
len(run["system"])

50000

In [9]:

from evaluate import load

def eval_ndcg10(run):
  trec_eval = load("trec_eval")
  results = trec_eval.compute(predictions=[run], references=[qrel])
  return results['NDCG@10'] 

Evaluates BM25 NDCG@10.

In [10]:
def search_all_bm25(top_k=1000):
  run = defaultdict(list)

  for id, query in tqdm(zip(query_ids, query_texts)):
    bm25_hits = search_with_bm25(query)
    ids = [json.loads(bm25_hits[i].raw)['_id'] for i in range(len(bm25_hits))]
    run["query"] += [id] * top_k
    run["docid"] += ids
    run["score"] += [1] * top_k
    run["q0"] += ["q0"] * top_k
    run["rank"] += list(range(1,top_k+1))
    run["system"] += [model_name] * top_k

  return run


In [ ]:
run_bm25 = search_all_bm25()

50it [00:05,  9.08it/s]


In [ ]:
eval_ndcg10(run_bm25)

0.5946917010118077

In [ ]:
eval_ndcg10(run)

0.2754485142056899

Conclusion: horrible quality for off-the-shelf usage.  It requiries fine tuning to be used on TREC-COVID.